# Educational Content Generator with Fact Verification Using Generative AI

## Introduction
This project addresses a critical challenge in education: creating accurate, grade-appropriate 
educational content with verified facts. Traditional content creation is time-consuming and may 
contain outdated or inaccurate information. Our solution leverages multiple generative AI 
capabilities to automate this process while ensuring accuracy and educational value.

### Problem Statement
Teachers and educational content creators face three main challenges:
1. Time-consuming content creation process
2. Difficulty in verifying factual accuracy
3. Ensuring grade-level appropriateness

### Solution Overview
Our system combines several generative AI capabilities to create an automated, reliable 
educational content generation system:
- Google Search grounding for fact verification
- Structured output for consistent content formatting
- Grade-level appropriate content generation
- Automated content quality assessment

In [ ]:
# Import required libraries
from google import genai
from google.genai import types
from IPython.display import Markdown, display
from typing import TypedDict, List
from datetime import datetime
# from kaggle_secrets import UserSecretsClient

# # Initialize Gemini client with API key
# client = genai.Client(api_key=UserSecretsClient().get_secret("GOOGLE_API_KEY"))

# Initialize the Gemini client
client = genai.Client(api_key="GOOGLE_API_KEY")

## Define Core Data Structures
We create type-safe structures to handle our educational content and analysis results.

In [4]:
class TestResults(TypedDict):
    """Structure to hold comprehensive test results"""
    timestamp: str
    content_metrics: dict
    fact_verification: dict
    grade_level_analysis: dict
    execution_time: float


## Content Generation and Analysis Implementation
The following section implements our core functionality for generating and analyzing educational content.

In [5]:
def execute_topic_test(topic: str):
    """
    Execute the main test with comprehensive logging and analysis for a user-specified topic.
    This function demonstrates the integration of multiple Gen AI capabilities:
    1. Content generation with fact verification
    2. Source grounding
    3. Educational content analysis

    Parameters:
        topic (str): The educational subject to generate content about.
    """
    
    start_time = datetime.now()
    display(Markdown(f"# Starting {topic.title()} Education Content Test"))
    display(Markdown(f"Test started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}"))

    try:
        # 1. Initialize Model Configuration
        config = types.GenerateContentConfig(
            tools=[types.Tool(google_search=types.GoogleSearch())],
            temperature=0.2,  # Lower temperature for more factual content
            top_p=0.8,
            top_k=40
        )

        # 2. Generate Content
        display(Markdown(f"## Generating Educational Content for {topic.title()}..."))
        prompt = f"""Create educational content about {topic} for 7th Grade students.
        Include:
        - Key scientific or conceptual ideas pertinent to the topic.
        - Real-world examples and impacts.
        - Recent data and statistics.
        - Age-appropriate explanations.
        - Actionable solutions.

        Ensure all facts are verified with reliable sources."""
        
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[prompt],
            config=config
        )

        # Document the generated content and its analysis
        display(Markdown("## Generated Content"))
        display(Markdown(response.text))

        # Analyze sources and grounding
        display(Markdown("## Source Analysis"))
        if hasattr(response, 'grounding_metadata') and response.grounding_metadata:
            chunks = response.grounding_metadata.grounding_chunks
            display(Markdown("### Verified Sources:"))
            for i, chunk in enumerate(chunks, 1):
                display(Markdown(f"""
                **Source {i}:**
                - **Title:** {chunk.web.title}
                - **URL:** {chunk.web.uri}
                - **Referenced Content:** "{chunk.text}"
                """))
        
        # Calculate comprehensive metrics
        results = TestResults(
            timestamp=start_time.strftime('%Y-%m-%d %H:%M:%S'),
            content_metrics={
                "total_length": len(response.text),
                "num_paragraphs": len(response.text.split('\n\n')),
                "reading_level_score": calculate_reading_level(response.text)
            },
            fact_verification={
                "verified_sources": len(response.grounding_metadata.grounding_chunks) if hasattr(response, 'grounding_metadata') and response.grounding_metadata else 0,
                "fact_density": calculate_fact_density(response.text)
            },
            grade_level_analysis=analyze_grade_appropriateness(response.text),
            execution_time=(datetime.now() - start_time).total_seconds()
        )

        # Display analysis results
        display_test_results(results)

    except Exception as e:
        display(Markdown(f"## Error During Test Execution\n```python\n{str(e)}\n```"))
        raise


## Analysis Helper Functions
These functions implement our educational content analysis capabilities.

In [6]:
def calculate_reading_level(text: str) -> float:
    """
    Calculate approximate reading level score using a simplified formula
    Based on average sentence length and complex word frequency.
    """
    words = text.split()
    sentences = text.split('.')
    avg_words_per_sentence = len(words) / len(sentences) if sentences else 0
    complex_words = len([w for w in words if len(w) > 8])
    return (avg_words_per_sentence * 0.39) + (complex_words / len(words) * 100 * 0.185) if words else 0

def calculate_fact_density(text: str) -> float:
    """
    Calculate the density of factual statements in the text.
    Uses linguistic markers to identify potential fact statements.
    """
    fact_indicators = ['is', 'are', 'was', 'were', 'according to', 'research shows']
    words = text.split()
    fact_indicators_count = sum(1 for word in words if any(indicator in word.lower() for indicator in fact_indicators))
    return fact_indicators_count / len(words) if words else 0

def analyze_grade_appropriateness(text: str) -> dict:
    """
    Analyze if content is appropriate for 7th grade level.
    Returns metrics for sentence complexity and vocabulary.
    """
    words = text.split()
    sentences = text.split('.')
    return {
        "avg_words_per_sentence": len(words) / len(sentences) if sentences else 0,
        "complex_word_ratio": len([w for w in words if len(w) > 8]) / len(words) if words else 0,
        "technical_terms": identify_technical_terms(text)
    }

def identify_technical_terms(text: str) -> list:
    """
    Identify scientific/technical terms in the content.
    Helps assess vocabulary complexity.
    """
    scientific_indicators = ['carbon', 'dioxide', 'greenhouse', 'atmosphere', 'temperature']
    return [word for word in text.lower().split() if any(term in word for term in scientific_indicators)]


## Results Display Function
This function provides a comprehensive view of the analysis results.

In [7]:
def display_test_results(results: TestResults):
    """Display comprehensive test results with detailed metrics"""
    display(Markdown(f"""
    # Test Results Summary
    
    ## Execution Metrics
    - **Test Duration:** {results['execution_time']:.2f} seconds
    - **Timestamp:** {results['timestamp']}
    
    ## Content Analysis
    - **Total Length:** {results['content_metrics']['total_length']} characters
    - **Number of Paragraphs:** {results['content_metrics']['num_paragraphs']}
    - **Reading Level Score:** {results['content_metrics']['reading_level_score']:.2f}
    
    ## Fact Verification
    - **Verified Sources:** {results['fact_verification']['verified_sources']}
    - **Fact Density:** {results['fact_verification']['fact_density']:.2%}
    
    ## Grade Level Appropriateness
    - **Average Words per Sentence:** {results['grade_level_analysis']['avg_words_per_sentence']:.1f}
    - **Complex Word Ratio:** {results['grade_level_analysis']['complex_word_ratio']:.2%}
    - **Technical Terms Found:** {len(results['grade_level_analysis']['technical_terms'])}
    """))


## Execution
Let's run our educational content generator and analysis system for a user-specified topic.

In [ ]:
# Set the topic for educational content generation
try:
    user_topic = "Climate Change"  # Replace this with any topic you desire
    execute_topic_test(user_topic)
except Exception as e:
    display(Markdown(f"# Test Failed\nError: {str(e)}"))
finally:
    display(Markdown("# Test Execution Completed"))


## Conclusion

This implementation demonstrates the effective integration of multiple generative AI capabilities:
1. Google Search grounding for fact verification
2. Structured output generation for educational content
3. Automated content analysis and evaluation

### Key Achievements:
- Automated generation of grade-appropriate educational content on any given topic
- Real-time fact verification with source citations
- Comprehensive content quality analysis

### Future Improvements:
1. Enhanced multi-subject support
2. Interactive, user-based content customization
3. Customizable curriculum alignment
4. Expanded evaluation metrics

### References:
1. Gemini API Documentation
2. Educational Content Standards
3. Reading Level Assessment Methodologies

This notebook successfully demonstrates the potential of generative AI in educational content creation,
providing a foundation for further advancements in automated educational content generation.